In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('kaggle/input/loan-payback/train.csv')
test = pd.read_csv('kaggle/input/loan-payback/test.csv')

In [4]:
train.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [7]:
# No nulls
train.isnull().sum()

id                      0
annual_income           0
debt_to_income_ratio    0
credit_score            0
loan_amount             0
interest_rate           0
gender                  0
marital_status          0
education_level         0
employment_status       0
loan_purpose            0
grade_subgrade          0
loan_paid_back          0
dtype: int64

In [8]:
train.describe()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,loan_paid_back
count,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000
mean,296996.500000,48212.202976,0.120696,680.916009,15020.297629,12.356345,0.798820
std,171471.442236,26711.942078,0.068573,55.424956,6926.530568,2.008959,0.400883
min,0.000000,6002.430000,0.011000,395.000000,500.090000,3.200000,0.000000
25%,148498.250000,27934.400000,0.072000,646.000000,10279.620000,10.990000,1.000000
50%,296996.500000,46557.680000,0.096000,682.000000,15000.220000,12.370000,1.000000
75%,445494.750000,60981.320000,0.156000,719.000000,18858.580000,13.680000,1.000000
max,593993.000000,393381.740000,0.627000,849.000000,48959.950000,20.990000,1.000000


In [ ]:
# Create interaction features between quantitative columns
quantitative_features = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
                         'loan_amount', 'interest_rate']

def create_interaction_features(df):
    """
    Create interaction features by combining quantitative features
    """
    df = df.copy()
    
    # Income-related ratios
    df['income_to_loan_ratio'] = df['annual_income'] / (df['loan_amount'] + 1)
    df['monthly_payment_estimate'] = (df['loan_amount'] * df['interest_rate'] / 100) / 12
    
    # Credit and risk interactions
    df['credit_to_loan_ratio'] = df['credit_score'] / (df['loan_amount'] + 1)
    df['risk_score'] = df['debt_to_income_ratio'] * df['interest_rate']
    
    # Affordability metrics
    df['monthly_income'] = df['annual_income'] / 12
    df['affordability_ratio'] = df['monthly_income'] / (df['monthly_payment_estimate'] + 1)
    
    # Composite scores
    df['credit_income_score'] = df['credit_score'] * (df['annual_income'] / 100000)
    df['debt_burden'] = df['annual_income'] * df['debt_to_income_ratio']
    
    return df

train = create_interaction_features(train)
test = create_interaction_features(test)

Index(['id', 'annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade', 'loan_paid_back'],
      dtype='object')

In [11]:
features = list(train.columns.drop(['id', 'loan_paid_back']))
features

['annual_income',
 'debt_to_income_ratio',
 'credit_score',
 'loan_amount',
 'interest_rate',
 'gender',
 'marital_status',
 'education_level',
 'employment_status',
 'loan_purpose',
 'grade_subgrade']